In [69]:
import requests
import pandas as pd
from my_api import api_key

pd.set_option('display.max_columns', None)

# https://data.mos.ru/ источник данных

# all_datests = requests.get(f'https://apidata.mos.ru/v1/datasets/?api_key={api_key}').json()

dataset_dict={ #словарь с индентификаторами датасетов на портале открытых данных
    'theaters':'531',
    'library':'526',
    'museum':'529',
    'wifi_bibl':'60788',
    'wifi_kino':'60789',
    'wifi_cult':'60790',
    'wifi_park':'861',
    'wifi_city':'2756',
    'food':'1903'
}
dataset_file_list = list(map(lambda x: x+'.pkl',list(dataset_dict))) # список сохраненных в будущем файлов

# цикл сохранения в пикл файлы
for each in dataset_dict:
    print(f'''Start {each}''')
    len_df = requests.get(f"https://apidata.mos.ru/v1/datasets/{dataset_dict[each]}/count?api_key={api_key}").json() #Получаем общее кол-во строк в наборе данных
    print(f'''Dataset length = {len_df}''')

    df_list = []

    i=0
    while i < len_df:
         url=f'https://apidata.mos.ru/v1/datasets/{dataset_dict[each]}/rows?api_key={api_key}&$top=1000&$skip={str(i)}'
         response = requests.get(url).json()
         df_list.append(pd.json_normalize(response))
         i+=1000
         print(f"{i-1000} - {i} done",end='\r')
    print('')
    if len(df_list) > 1:
        pd.concat(df_list).to_pickle(f'{each}.pkl')
    else:
        df_list[0].to_pickle(f'{each}.pkl')


final_df = [] # список, который будет превращен затем в датафрейм

#форматирование датафреймов под единый стандарт
for each in dataset_file_list:
    x = pd.read_pickle(each)
    x['data_type'] = x['global_id'].apply(lambda x: each)

    if 'Cells.WebSite' not in x.columns:
        x['Cells.WebSite'] = x['global_id'].apply(lambda x: '-')
    if 'Cells.Latitude_WGS84' not in x.columns:
        x['Cells.Latitude_WGS84'] = x['Cells.geoData.coordinates'].apply(lambda x: x[0][1])
    if 'Cells.Longitude_WGS84' not in x.columns:
        x['Cells.Longitude_WGS84'] = x['Cells.geoData.coordinates'].apply(lambda x: x[0][0])
    if 'Cells.CommonName' in x.columns:
        x['name'] = x['Cells.CommonName']
    elif 'Cells.WiFiName' in x.columns:
        x['name'] = x['Cells.WiFiName']
    else:
        x['name'] = x['Cells.Name']

    final_df.append(x[['data_type','global_id','Cells.Latitude_WGS84','Cells.Longitude_WGS84','Cells.WebSite','name']]) # выбираем колонки и присоденияем к списку датафрейм

# получаем итоговый датафрейм 
df = pd.concat(final_df)
df.rename(columns={'Cells.Latitude_WGS84':'lat',
                   'Cells.Longitude_WGS84':'lon'},inplace=True)
df['data_type'] = df['data_type'].apply(lambda x: x[0:-4])


df.to_excel('tourist.xlsx',index=False) #сохраняем, чтобы вручную потом скопировать в google_sheets


Start theaters
Dataset length = 67
0 - 1000 done
Start library
Dataset length = 271
0 - 1000 done
Start museum
Dataset length = 52
0 - 1000 done
Start wifi_bibl
Dataset length = 360
0 - 1000 done
Start wifi_kino
Dataset length = 11
0 - 1000 done
Start wifi_cult
Dataset length = 370
0 - 1000 done
Start wifi_park
Dataset length = 1126
1000 - 2000 done
Start wifi_city
Dataset length = 3108
3000 - 4000 done
Start food
Dataset length = 20933
20000 - 21000 done
